In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")


In [ ]:
from langchain_core.messages import HumanMessage

model.invoke([HumanMessage(content="Hi! I'm Bob")])

In [ ]:
model.invoke([HumanMessage(content="What's my name?")])

In [ ]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph

# Define a new graph
workflow = StateGraph(state_schema=MessagesState)


# Define the function that calls the model
def call_model(state: MessagesState):
    response = model.invoke(state["messages"])
    return {"messages": response}


# Define the (single) node in the graph
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

# Add memory
memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
config = {"configurable": {"thread_id": "abc123"}}

In [ ]:
query = "Hi! I'm Bob."

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()  # output contains all messages in state

In [ ]:
query = "What's my name?"

input_messages = [HumanMessage(query)]
output = app.invoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Async

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import HumanMessage

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

config = {"configurable": {"thread_id": "abc123"}}


In [ ]:
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver

# Async function for node:
async def call_model(state: MessagesState):
    response = await model.ainvoke(state["messages"])
    return {"messages": response}


# Define graph as before:
workflow = StateGraph(state_schema=MessagesState)
workflow.add_edge(START, "model")
workflow.add_node("model", call_model)
aapp = workflow.compile(checkpointer=MemorySaver())



In [ ]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()


In [ ]:

query = "What's my name?"
input_messages = [HumanMessage(query)]

# Async invocation:
output = await aapp.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Adding a System Message

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You talk like a pirate. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

In [ ]:
workflow = StateGraph(state_schema=MessagesState)


def call_model(state: MessagesState):
    prompt = prompt_template.invoke(state)
    response = model.invoke(prompt)
    return {"messages": response}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

memory = MemorySaver()
app = workflow.compile(checkpointer=memory)

In [ ]:
query = "Hi! I'm Bob."
input_messages = [HumanMessage(query)]

# Async invocation:
output = await app.ainvoke({"messages": input_messages}, config)
output["messages"][-1].pretty_print()

# Trimming

In [ ]:
from langchain.chat_models import init_chat_model

model = init_chat_model("deepseek-r1:32b", model_provider="ollama")
model = init_chat_model("llama3.3", model_provider="ollama")

In [ ]:
from langchain_core.messages import SystemMessage, AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph import START, MessagesState, StateGraph
from langgraph.graph.message import add_messages


trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)


workflow = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}


workflow.add_edge(START, "model")
workflow.add_node("model", call_model)

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.checkpoint.sqlite import SqliteSaver
import sqlite3
import aiosqlite


db_path = 'checkpoints.db'
conn = sqlite3.connect(db_path, check_same_thread=False)

#memory = SqliteSaver(conn)
# Does not work
#memory = AsyncSqliteSaver(conn)


memory = MemorySaver()
app = workflow.compile(checkpointer=memory)
        
print(type(app))

In [ ]:
from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="What is 1+1?")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)
    async for event in graph.astream_events(inputs, config, version="v1"):
        pprint(event)

In [ ]:
# THIS COULD BE THE ONE I NEED

from pprint import pprint

config = {"configurable": {"thread_id": "abc12"}}
inputs = {"messages": [HumanMessage(content="Hi! I'm Bob.")]}

async with AsyncSqliteSaver.from_conn_string(db_path) as saver:
    graph = workflow.compile(checkpointer=saver)        
    async for chunk, metadata in graph.astream(
        inputs,
        config,
        stream_mode="messages",
    ):
        if isinstance(chunk, AIMessage):  # Filter to just model responses
            print(chunk.content, end="")

In [ ]:
import json


config = {"configurable": {"thread_id":  "abc12"}}



with SqliteSaver.from_conn_string(db_path) as checkpointer:
    graph = workflow.compile(checkpointer=checkpointer)
    state_history = graph.get_state_history(config) 
    values = next(state_history).values  
    print(values) 
    print(len(values['messages']))
    for message in values['messages']:
        if isinstance(message, HumanMessage):
            print(f"Human: {message.content}")
        else:
            print(f"AI:    {message.content}")
    
    #state_history_list = list(state_history)
    #print(state_history_list)
    # for state in state_history:
    #     print(state)
    #     # print(f"Step: {state.step}")
    #     # print(f"Created At: {state.created_at}")
    #     # print(f"Values: {state.values}")
    #     # print("-----")
 
    #list(graph.get_state_history(config))

In [ ]:
query = "What is my name"
input_messages = [HumanMessage(query)]

async for chunk, metadata in app.astream(
    {"messages": input_messages},
    config,
    stream_mode="messages",
):
    if isinstance(chunk, AIMessage):  # Filter to just model responses
        print(chunk.content, end="|")

In [ ]:
import asyncio

from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import StateGraph

builder = StateGraph(int)
builder.add_node("add_one", lambda x: x + 1)
builder.set_entry_point("add_one")
builder.set_finish_point("add_one")
async with AsyncSqliteSaver.from_conn_string("checkpoints.db") as memory:
    graph = builder.compile(checkpointer=memory)
    coro = graph.ainvoke(1, {"configurable": {"thread_id": "thread-1"}})
    print(asyncio.run(coro))

# Async history

In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.checkpoint.sqlite.aio import AsyncSqliteSaver
from langgraph.graph import START, MessagesState, StateGraph



system_prompt = "Answer all questions to the best of your ability. Answer concisely but correctly. If you do not know the answer, just say 'I don’t know.'"
model = init_chat_model("deepseek-r1:32b", model_provider="ollama")

trimmer = trim_messages(
    max_tokens=6_500_000,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human",
)

prompt_template = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            system_prompt,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

graph = StateGraph(state_schema=MessagesState)

def call_model(state: MessagesState):
    trimmed_messages = trimmer.invoke(state["messages"])
    prompt = prompt_template.invoke({"messages": trimmed_messages})
    response = model.invoke(prompt)
    return {"messages": [response]}

graph.add_edge(START, "model")
graph.add_node("model", call_model)



In [ ]:
print("IN get_chat_interactions_count")
interactions_count = 0
db_path="/Users/dan/Code/LLM/DeepDive/backend/resources/checkpoints.db"
chat_id="ea9123ff-9a8e-46c0-a53f-22b8f88e3202"

with SqliteSaver.from_conn_string(db_path) as checkpointer:            
    config = {"configurable": {"thread_id": chat_id}}
    print("before compiled_graph")
    compiled_graph = graph.compile(checkpointer=checkpointer)
    print("after compiled_graph")
    state_history = compiled_graph.get_state_history(config) 
    print("after state_history")
    last_interaction = next(state_history, None)
    print("after last_interaction")
    if last_interaction:
        values = last_interaction.values  
        if 'messages' in values:
            interactions_count = len(values['messages'])
            
interactions_count

In [ ]:
import sys
import os
sys.path.append('/Users/dan/Code/LLM/DeepDive/backend')

import app.app_session
file_path = app.app_session.__file__
parent_directory = os.path.dirname(file_path)
grandparent_directory = os.path.dirname(parent_directory)
grandparent_directory

# GET CONVERSATION HISTORY


In [ ]:
from langchain.chat_models import init_chat_model
from langchain_core.messages import AIMessage, HumanMessage, trim_messages
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.graph import START, MessagesState, StateGraph

model = init_chat_model("llama3.2:1b", model_provider="ollama")
system_prompt = "Do your best to answer correctly"

def create_graph():
    #TODO THIS DOES NOT LOOK RIGHT    
    trimmer = trim_messages(
        max_tokens=6_500_000,
        strategy="last",
        token_counter=model,
        include_system=True,
        allow_partial=False,
        start_on="human",
    )

    prompt_template = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                system_prompt,
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )

    graph = StateGraph(state_schema=MessagesState)

    def call_model(state: MessagesState):
        trimmed_messages = trimmer.invoke(state["messages"])
        prompt = prompt_template.invoke({"messages": trimmed_messages})
        response = model.invoke(prompt)
        return {"messages": [response]}

    graph.add_edge(START, "model")
    graph.add_node("model", call_model)

    return graph


graph = create_graph()
        

In [ ]:

db_path = "/Users/dan/Code/LLM/DeepDive/backend/resources/checkpoints.db"

def get_chat_interactions_count(chat_id):
    interactions_count = 0
    with SqliteSaver.from_conn_string(db_path) as checkpointer:            
        config = {"configurable": {"thread_id": chat_id}}
        compiled_graph = graph.compile(checkpointer=checkpointer)
        state_history = compiled_graph.get_state_history(config) 
        last_interaction = next(state_history, None)
        if last_interaction:
            values = last_interaction.values  
            if 'messages' in values:
                interactions_count = len(values['messages'])
    return interactions_count

In [ ]:
def get_chat_interactions(chat_id):
    interactions=[]
    with SqliteSaver.from_conn_string(db_path) as checkpointer:            
        config = {"configurable": {"thread_id": chat_id}}
        compiled_graph = graph.compile(checkpointer=checkpointer)
        state_history = compiled_graph.get_state_history(config) 
        last_interaction = next(state_history, None)
        if last_interaction:
            values = last_interaction.values  
            if 'messages' in values:
                for message in values['messages']:
                    #print(type(message))
                    #print(message.content)
                    interaction_type = 'user' if isinstance(message,HumanMessage) else 'ai'
                    interaction = {'type':interaction_type, 'content':message.content}
                    interactions.append(interaction)
    return interactions

In [ ]:
chat_id="ea9123ff-9a8e-46c0-a53f-22b8f88e3202"
get_chat_interactions(chat_id)


In [4]:
import requests
from requests.exceptions import RequestException
import time

def searxng_search(query, max_results=5, retries=3, timeout=10):
    searxng_url = "https://searxng.danhiggins.org/search"
    
    # Define your search query parameters
    params = {
        "q": query,
        "format": "json"
    }
    
    # Initialize variables for retry logic
    attempt = 0
    last_exception = None
    
    while attempt < retries:
        try:
            # Send the request to the SearxNG instance with timeout
            response = requests.get(searxng_url, params=params, timeout=timeout)
            
            # Check if the request was successful
            if response.status_code == 200:
                # Parse the JSON response
                json_response = response.json()
                
                # Get results with a fallback to an empty list
                json_response_results = json_response.get("results", [])
                
                # Limit results if needed
                json_response_results = json_response_results[:max_results]
                
                # Map the response
                json_response_mapped = [
                    {
                        'href': item.get('url', ''),
                        'title': item.get('title', ''),
                        'body': item.get('content', '')
                    }
                    for item in json_response_results
                    if any(key in item for key in ('url', 'title', 'content'))
                ]
                        
                return json_response_mapped
            else:
                last_exception = f"Error: HTTP {response.status_code}"
                
        except requests.exceptions.Timeout:
            last_exception = f"Timeout after {timeout} seconds"
        except RequestException as e:
            last_exception = f"Request failed: {str(e)}"
        
        # Increment attempt count and wait before retrying (with exponential backoff)
        attempt += 1
        if attempt < retries:
            wait_time = min(2 ** attempt, 30)  # Max wait time of 30 seconds
            print(f"Attempt {attempt} failed: {last_exception}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    
    # If all retries failed, raise an exception with the last error
    raise Exception(f"All {retries} attempts failed. Last error: {last_exception}")


In [5]:
query ="who is bernie sanders"
json_response = searxng_search(query)
print(len(json_response))
json_response

5


[{'href': 'https://en.wikipedia.org/wiki/Bernie_Sanders',
  'title': 'Bernie Sanders - Wikipedia',
  'body': '3 days ago - Bernard Sanders was born on September 8, 1941, in the Brooklyn borough of New York City. His father, Elias Ben Yehuda Sanders (1904–1962), a Polish-Jewish immigrant, was born in Słopnice, a town in Austrian Galicia that was then part of the Austro-Hungarian Empire and is now in Poland.'},
 {'href': 'https://www.sanders.senate.gov/about-bernie/',
  'title': 'About Bernie - Senator Bernie Sanders',
  'body': 'Bernie Sanders is serving his fourth term in the U.S. Senate after winning re-election in 2024. His previous 16 years in the House of Representatives make him the longest serving independent member of Congress in American history. Born in 1941 in Brooklyn, Sanders attended James Madison High ...'},
 {'href': 'https://www.britannica.com/biography/Bernie-Sanders',
  'title': 'Bernie Sanders | Biography & Facts | Britannica',
  'body': '2 weeks ago - Bernie Sanders

In [1]:
import time
import logging
from duckduckgo_search import DDGS
from duckduckgo_search.exceptions import DuckDuckGoSearchException, RatelimitException, TimeoutException


def ddg_search(query, max_results=5, retries=3, timeout=10):
    """Perform a search using DuckDuckGo's API with retry and timeout handling."""
    search_response = []
    attempt = 0
    while attempt < retries:
        try:
            ddg = DDGS(timeout=timeout)
            search_response = ddg.text(
                query,
                region="wt-wt",
                safesearch="off",
                timelimit="y",
                backend="lite",
                max_results=max_results,
            )
            break  # Exit loop if search is successful
        except RatelimitException as e:
            attempt += 1
            wait_time = min(2 ** attempt, 30)  # Exponential backoff with a max wait time of 30 seconds
            #logger.debug(f"Rate limit exceeded: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except TimeoutException as e:
            attempt += 1
            wait_time = min(2 ** attempt, 30)
            #logger.debug(f"Timeout error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
        except DuckDuckGoSearchException as e:
            attempt += 1
            wait_time = min(2 ** attempt, 30)
            #logger.debug(f"DuckDuckGo search error: {e}. Retrying in {wait_time} seconds...")
            time.sleep(wait_time)
    else:
        #logger.debug("Maximum retries reached. Returning empty search response.")
        pass
    return search_response


In [2]:
query ="who is bernie sanders"
ddg_search(query, max_results=5, retries=3, timeout=10)

[{'title': 'Drawing huge crowds, Bernie Sanders emerges as the leader of the anti ...',
  'href': 'https://www.pbs.org/newshour/politics/drawing-huge-crowds-bernie-sanders-emerges-as-the-leader-of-the-anti-trump-resistance',
  'body': "WARREN, Mich. (AP) — Bernie Sanders is standing alone on the back of a pickup truck shouting into a bullhorn. He's facing several hundred ecstatic voters huddled outside a suburban Detroit ..."},
 {'title': 'Bernie Sanders takes leadership of the anti-Trump resistance - AP News',
  'href': 'https://apnews.com/article/bernie-sanders-democrats-trump-c213d5ae42737c956d46f6f7f17e5abd',
  'body': "WARREN, Mich. (AP) — Bernie Sanders is standing alone on the back of a pickup truck shouting into a bullhorn. He's facing several hundred ecstatic voters huddled outside a suburban Detroit high school — the group that did not fit inside the high school's gym or two overflow rooms. The crowd screams in delight when he tells them that a ..."},
 {'title': 'Bernie Sande